In [1]:
import itertools
import pandas
import numpy
import glob
import tqdm


In [2]:
# settings
protocols = ['fotb', 'totb', 'htlltb', 'htlltbtest']
n_processes = [3, 6, 9, 12]
prefixes = [str(a) + "-" + str(b) for a,b in itertools.product(protocols, n_processes)]


In [3]:
# check data
print("number of log files: %s" % len(glob.glob('../data/*.log')))
print("expected number of log files: %s" % (len(protocols) * sum(n_processes)))


number of log files: 120
expected number of log files: 120


In [4]:
# tests
# given two histories, h1 and h2, check if property is satisfied
# history is a dataframe with columns: operation, processid, messagenumber (or messageID), time


# run a test-function on all combinations of histories
# returns true, if property of f holds on all combinations of histories
def run_test(histories, f):
    for h1, h2 in itertools.combinations(histories, 2):
        if not f(h1, h2):
            return False
    return True


# total-order: both histories share same order of delivery 
# check: history of delivered messages of h1 is prefix of h2, or vice versa
def total_order(h1, h2):
    # test neither history is empty
    if len(h1) == 0 or len(h2) == 0:
        raise Exception({"error": "empty history"})
        
    # filter histories
    h1_filtered = h1[h1["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    h2_filtered = h2[h2["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    
    # h2_filtered shorter than h1_filtered
    if len(h1_filtered) < len(h2_filtered):
        h1_filtered, h2_filtered = h2_filtered, h1_filtered
    
    h1_filtered = h1_filtered.head(len(h2_filtered))
    
    # reset index
    h1_filtered = h1_filtered.reset_index(drop=True)
    h2_filtered = h2_filtered.reset_index(drop=True)
    
    # test if h1_filtered is equal to h2_filtered
    return h1_filtered.equals(h2_filtered) 


# fifo-order: messages of a process are delivered in the order they were broadcast by process
# check: broadcast messages of h1 are delivered in same order in h2, and vice versa
def fifo_order(h1, h2):
    # test neither history is empty
    if len(h1) == 0 or len(h2) == 0:
        raise Exception({"error": "empty history"})
        
    # filter histories
    h1_broadcast_filtered = h1[h1["operation"] == "broadcast"].filter(["operation", "processid", "messagenumber"])
    h1_process_id = h1.iloc[0]["processid"]
    h2_broadcast_filtered = h2[h2["operation"] == "broadcast"].filter(["operation", "processid", "messagenumber"])
    h2_process_id = h2.iloc[0]["processid"]
    h1_delivered_filtered = h1[h1["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    h1_delivered_filtered = h1_delivered_filtered[h1_delivered_filtered["processid"] == h2_process_id]
    h2_delivered_filtered = h2[h2["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    h2_delivered_filtered = h2_delivered_filtered[h2_delivered_filtered["processid"] == h1_process_id]

    # adjust lengths
    h1_broadcast_filtered = h1_broadcast_filtered.head(len(h2_delivered_filtered))
    h2_broadcast_filtered = h2_broadcast_filtered.head(len(h1_delivered_filtered))

    # remove operation and reset index
    h1_broadcast_filtered = h1_broadcast_filtered.drop("operation", axis=1).reset_index(drop=True)
    h2_broadcast_filtered = h2_broadcast_filtered.drop("operation", axis=1).reset_index(drop=True)
    h1_delivered_filtered = h1_delivered_filtered.drop("operation", axis=1).reset_index(drop=True)
    h2_delivered_filtered = h2_delivered_filtered.drop("operation", axis=1).reset_index(drop=True)
    
    # test if broadcast and delivered in same order
    return h1_broadcast_filtered.equals(h2_delivered_filtered) and h2_broadcast_filtered.equals(h1_delivered_filtered)


# no-duplication: messages are only delivered once
# check: no message is delivered twice in any history
def no_duplication(h1, h2):
    # test neither history is empty
    if len(h1) == 0 or len(h2) == 0:
        raise Exception({"error": "empty history"})
        
    # filter histories
    h1_delivered_filtered = h1[h1["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    h2_delivered_filtered = h2[h2["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    
    # test for no duplicates
    return not h1_delivered_filtered.duplicated().any() and not h2_delivered_filtered.duplicated().any()


# no-creation: only messages that have been broadcast are delivered
# check: if a message was delivered in h2 by p1, then it was previously broadcast in h1 by p1
def no_creation(h1, h2):
    # test neither history is empty
    if len(h1) == 0 or len(h2) == 0:
        raise Exception({"error": "empty history"})
        
    # filter histories
    h1_broadcast_filtered = h1[h1["operation"] == "broadcast"].filter(["operation", "processid", "messagenumber"])
    h1_process_id = h1.iloc[0]["processid"]
    h2_broadcast_filtered = h2[h2["operation"] == "broadcast"].filter(["operation", "processid", "messagenumber"])
    h2_process_id = h2.iloc[0]["processid"]
    h1_delivered_filtered = h1[h1["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    h1_delivered_filtered = h1_delivered_filtered[h1_delivered_filtered["processid"] == h2_process_id]
    h2_delivered_filtered = h2[h2["operation"] == "deliver"].filter(["operation", "processid", "messagenumber"])
    h2_delivered_filtered = h2_delivered_filtered[h2_delivered_filtered["processid"] == h1_process_id]

    # test if delivered of h1 was broadcast by h2, and vice versa
    h1_check = pandas.merge(h1_delivered_filtered, h2_broadcast_filtered, how='left', on=["processid", "messagenumber"])
    h2_check = pandas.merge(h2_delivered_filtered, h1_broadcast_filtered, how='left', on=["processid", "messagenumber"])
    return not h1_check.isna().any(axis=1).any() and not h2_check.isna().any(axis=1).any()


In [5]:
results = []

i = 0
for prefix in tqdm.tqdm(prefixes):
    files = glob.glob('../data/' + prefix + '*')
    dfs = []
    for file in files:
        d = pandas.read_csv(files[0], sep=' ', comment="{") 
        dfs.append(d)
    result = {
        "prefix": prefix,
        "total-order": run_test(dfs, total_order),
        "fifo-order": run_test(dfs, fifo_order),
        "no-duplication": run_test(dfs, no_duplication),
        "no-creation": run_test(dfs, no_creation),
    }
    results.append(result)
    
results = pandas.DataFrame(results)
results


100%|██████████| 16/16 [37:43<00:00, 141.48s/it]


,prefix,total-order,fifo-order,no-duplication,no-creation
0,fotb-3,True,True,True,True
1,fotb-6,True,True,True,True
2,fotb-9,True,True,True,True
3,fotb-12,True,True,True,True
4,totb-3,True,True,True,True
5,totb-6,True,True,True,True
6,totb-9,True,True,True,True
7,totb-12,True,True,True,True
8,htlltb-3,True,True,True,True
9,htlltb-6,True,True,True,True
